In [1]:
import tqdm
import json

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

## User-based Collaborative Filtering

#### Основная идея: 
Рекомендовать пользователю треки, которые понравились похожим на него пользователям

$$\hat r_{ui} = h^{-1} \left( \frac{\sum_{v \in N_i(u)} w_{uv} h(r_{vi})}{\sum_{v \in N_i(u)} w_{uv}} \right)$$

$N_i(u)$ - соседи пользователя $u$, которые оценили айтем $i$,
$w_{uv}, w_{ij}$ - веса соседей, 
$h$ - функция нормализации



**Нормализация**: В качестве функции нормализации используем среднее время прослушивания

**Веса**: Похожих пользователей будем искать по *cosine similarity*

**Отсутствующие данные**: заполним средним времнем прослушивания по пользователю

**Соседи**: в качестве соседей будем рассматривать всех пользователей. Q: Как это упростит формулу?

In [4]:
BOTIFY_DATA_DIR = "/Users/kliakhnovich/DEV/MADE/rec_sys_2023/recsys-itmo-spring-2023/botify/data"

data = pd.read_json("/tmp/log/data.json", lines=True)[["user", "time", "track"]].copy()

data.head()

,user,time,track
0,3255,1.00,2659
1,3255,0.74,5125
2,3255,1.00,789
3,3255,0.07,740
4,3255,0.12,20379


In [5]:
data["normalized_time"] = data.groupby("user")["time"].transform(lambda time: time - time.mean())

data.head()

,user,time,track,normalized_time
0,3255,1.00,2659,0.727059
1,3255,0.74,5125,0.467059
2,3255,1.00,789,0.727059
3,3255,0.07,740,-0.202941
4,3255,0.12,20379,-0.152941


In [6]:
data.time.mean()


0.41421468016206997

In [7]:
interactions = pd.pivot_table(data, values="normalized_time", index="user", columns="track").fillna(0)

print(f"Interactions matrix: shape={interactions.shape}, sparsity={(interactions != 0).values.sum() / interactions.size}")

Interactions matrix: shape=(7113, 28774), sparsity=0.0005470869904372935


In [11]:
similarity_matrix = cosine_similarity(interactions)
np.fill_diagonal(similarity_matrix, 0)

print(f"Mean positive neighbours per user: {(similarity_matrix > 0).sum(axis=1).mean()}")

Mean positive neighbours per user: 340.79319555743007


In [13]:
print(f"Mean negative neighbours per user: {(similarity_matrix < 0).sum(axis=1).mean()}")

Mean negative neighbours per user: 314.978771263883


In [14]:
# TODO: Compute proper user-based scores
# TODO: expected size: observed users x observed tracks
scores_matrix = np.matmul(similarity_matrix, interactions.values)

scores = pd.DataFrame(
    scores_matrix,
    index=interactions.index,
    columns=interactions.columns
)

scores[[1, 2, 3, 4, 5]].head()

KeyError: '[1, 3, 4] not in index'

## Глянем на рекомендации

In [16]:
products = pd.read_json(BOTIFY_DATA_DIR + "tracks.json", lines=True).set_index("track")
products.head()

FileNotFoundError: File /Users/kliakhnovich/DEV/MADE/rec_sys_2023/recsys-itmo-spring-2023/botify/datatracks.json does not exist

In [46]:
user = np.random.choice(scores.index)
k = 10

# data[data["user"] == user]

In [47]:
data[data["user"] == user]

,user,time,track,normalized_time
107149,9071,1.00,3918,0.828333
107152,9071,0.03,48975,-0.141667
107155,9071,0.00,46220,-0.171667
107157,9071,0.00,14204,-0.171667
107159,9071,0.00,5984,-0.171667
107161,9071,0.00,39056,-0.171667


In [49]:
user_scores = pd.merge(
    scores.loc[user].sort_values(ascending=False)[:k].to_frame("score"),
    products, 
    left_index=True, 
    right_index=True,
    how="inner"
)

user_scores

,score,artist,title
track,,,
3918,2.474152,Mike And The Mechanics,A Beggar On A Beach Of Gold
38988,0.456922,Cabaret Voltaire,Do The Mussolini (Head Kick) They Kill Him Dub
38763,0.355865,Hole,Rock Star
14754,0.348692,Mogwai,Glasgow Mega-Snake
14401,0.327570,Katie Melua,On The Road Again
3590,0.313707,The Ruts,West One (Shine On Me)
1487,0.297004,Justin Bieber,One Time
1495,0.297004,Justin Bieber,U Smile
8262,0.297004,The Prodigy,The Big Gundown


In [48]:
user_interactions = pd.merge(
    interactions.loc[user].sort_values(ascending=False).to_frame("time"),
    products, 
    left_index=True, 
    right_index=True, 
    how="inner"
)

user_interactions[user_interactions["time"] != 0]

,time,artist,title
track,,,
3918,0.828333,Mike And The Mechanics,A Beggar On A Beach Of Gold
48975,-0.141667,Blitzen Trapper,Summer Town (Album)
46220,-0.171667,Third Day,Believe
14204,-0.171667,Valley of the Giants,Bala Bay Inn
39056,-0.171667,Marshall Jefferson,Move Your Body
5984,-0.171667,Bad Company,Gone_ Gone_ Gone


## Подготавливаем рекомендации для продакшена

In [ ]:
def recommend(user_id, scores, k):
    return scores.loc[user_id].sort_values(ascending=False)[:k].index.tolist()

In [ ]:
users = data["user"].unique()

with open(BOTIFY_DATA_DIR + "recommendations_ub.json", "w") as rf:
    for user in tqdm.tqdm(users):
        recommendation = {
            "user": int(user),
            "tracks": recommend(user, scores, 100)
        }
        rf.write(json.dumps(recommendation) + "\n")